In [2]:
from platform import python_version
print(python_version())

3.8.7


In [3]:
#https://crime-data-explorer.app.cloud.gov/explorer/state/georgia/crime

In [4]:
import pandas as pd
import numpy as np, sys, time, pdb, matplotlib.pyplot as plt
from pymongo import MongoClient

In [40]:
pd.set_option('display.max_rows', 100)# None

In [41]:
data_2020 = pd.read_csv('./flask/static/data/COBRA-2020.csv')
data_2021 = pd.read_csv('./flask/static/data/COBRA-2021.csv')
data_2009_2019 = pd.read_csv('./flask/static/data/COBRA-2009-2019.csv', low_memory=False)

In [ ]:
data_2009_2019.describe()

In [ ]:
data_2020_list = data_2020.columns.values.tolist()
data_2021_list = data_2021.columns.values.tolist()
data_2009_2019_list = data_2009_2019.columns.values.tolist()

In [ ]:
print(len(data_2020_list))
data_2020.head(1)

In [ ]:
print(len(data_2009_2019_list))
data_2009_2019.tail(10)

In [ ]:
#Since the 2009-2019 dataset is the largest, everything will try and have the same columns as that dataset
data_2020.rename(columns = {
 'offense_id' : 'Report Number',
 'rpt_date' : 'Report Date',
 'occur_date' : 'Occur Date',
 'occur_time' : 'Occur Time',
 'poss_date' : 'Possible Date',
 'poss_time' : 'Possible Time',
 'beat' : 'Beat',
 'apt_office_prefix' : 'Apartment Office Prefix',
 'apt_office_num' : 'Apartment Number',
 'location' : 'Location',
 #'MinOfucr' : ,
 #'dispo_code' : ,
 'Shift' : 'Shift Occurence',
 'loc_type' : 'Location Type',
 'UC2_Literal': 'UCR Literal',
 'ibr_code' : 'IBR Code',
 'neighborhood' : 'Neighborhood',
 'npu' : 'NPU',
 'long' : 'Longitude',
 'lat' : 'Latitude'
}, inplace = True)
#Drop columns that do not seem to show up in 2009-2019 dataset
data_2020 = data_2020.drop(columns=['MinOfucr','dispo_code'])
data_2020.head(1)

In [ ]:
list1 = data_2009_2019.columns.values.tolist()
list2 = data_2020.columns.values.tolist()

list_difference = [item for item in list1 if item not in list2]
print(list_difference)

In [ ]:
#It can be seen that the 'UCR #' shows up only in the 2009-2019 data and realistically serves no purpose because its a code only a cop would udnerstand
#So it should be droped to simplify
data_2009_2019 = data_2009_2019.drop(columns=['UCR #'])

In [ ]:
list1 = data_2009_2019.columns.values.tolist()
list2 = data_2020.columns.values.tolist()

list_difference = [item for item in list1 if item not in list2]
print(list_difference)
#There should now be no differences as at all

In [ ]:
#data[['Report Date','Occur Date','Possible Date']] = data[['Report Date','Occur Date','Possible Date']].apply(pd.to_datetime)

In [ ]:
#Concat the dataframes together
data = pd.concat([data_2009_2019, data_2020])

In [ ]:
data_2009_2019['Occur Date'].sort_values()

In [ ]:
data['Report Date'] = pd.to_datetime(data['Report Date'])
#data['Occur Date'] = pd.to_datetime(data['Occur Date'])

In [ ]:
#data = data.astype({'Report Date':'datetime64','Occur Date':'datetime64','Possible Date':'datetime64'},errors = 'raise')

In [ ]:
data.dtypes

In [ ]:
data.describe

In [ ]:
data.to_csv('data.csv')

In [ ]:
df.reset_index(drop=True, inplace=True)
df.to_json('data.json')

# Machine Learnings

## Read the CSV and Perform Basic Data Cleaning

In [22]:
import pandas as pd
import numpy as np

In [23]:
df = pd.read_csv("data.csv")
# Drop the null columns where all values are null
#df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df[["Latitude","Longitude","Neighborhood"]]
df = df.dropna()


/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (4,6,7,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [24]:
#df  = df.head(100000)

## Create a Train Test Split

Use `Neighborhood` for the y values
This can be an idiot check to see if lat an lon can predict neighborhood

In [26]:
from sklearn.model_selection import train_test_split
y = df["Neighborhood"]
X = df.drop(columns=["Neighborhood"])

#For example, if variable y is a binary categorical variable with values 0 and 1 and there are 25% of zeros and 75% of ones, stratify=y will make sure that your random split has 25% of 0's and 75% of 1's.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

# Pre-processing

Scale the data using the MinMaxScaler

In [27]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)#Compute the minimum and maximum to be used for later scaling.

X_train_scaled = X_scaler.transform(X_train)#Scale features of X_train according to feature_range.
X_test_scaled = X_scaler.transform(X_test)

In [33]:
y_train.nunique()

243

# Train the Model

In [7]:
from sklearn.linear_model import LogisticRegression

In [29]:
model = LogisticRegression(max_iter=200)
model.fit(X_train_scaled, y_train)# Fit the LR model

/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=200)

In [37]:
df

,Latitude,Longitude,Neighborhood
0,33.688450,-84.493280,Greenbriar
1,33.753200,-84.392010,Downtown
2,33.757350,-84.502820,Adamsville
3,33.846760,-84.362120,Lenox
4,33.686770,-84.497730,Greenbriar
...,...,...,...
350158,33.747960,-84.400728,Castleberry Hill
350159,33.737731,-84.368560,Grant Park
350160,33.742222,-84.341593,East Atlanta
350161,33.772028,-84.415209,English Avenue


In [30]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.3928318740603166
Testing Data Score: 0.3916935259420199


In [ ]:
model.predict()

In [ ]:
X_scaler.transform(X_train)[0]

In [34]:
model.predict(X_train_scaled)

array(['Downtown', 'Downtown', 'Grove Park', ..., 'Buckhead Forest',
       'Midtown', 'East Atlanta'], dtype=object)

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [49]:
from sklearn.model_selection import GridSearchCV
# Cfloat, default=1.0 Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.
# penalty{‘l1’, ‘l2’, ‘elasticnet’, ‘none’}, default=’l2’
param_grid = {'C': [1, 3,5,7,9],
              'penalty': ["l1", "l2", "none"]}
Model = LogisticRegression(solver='liblinear')
grid = GridSearchCV(Model, param_grid, verbose=3)
grid

NameError: name 'LogisticRegression' is not defined

In [ ]:
idk = grid.fit(X_train_scaled, y_train)
idk

# Need to Decrease Sample Size

It has become apparent that there is too mach data and our relatively weak computers are not up to the challenge give the time.

It was decided to just look at 5 major "Neighborhood" of interest:

buckhead, midtown, brookhaven, downtown, old fourth ward

In [71]:
df = pd.read_csv("data.csv")
# Drop the null columns where all values are null
#df = df.dropna(axis='columns', how='all')
# Drop the null rows
df_reduced = df[["Latitude","Longitude","Neighborhood"]]
df_reduced = df_reduced.dropna()

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (4,6,7,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [72]:
df_reduced["Neighborhood"].unique();

In [73]:
#Used to show all rows for error checking
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', 100)

In [74]:
df_buckhead = df_reduced[(df_reduced["Neighborhood"] == "Buckhead Forest") |(df_reduced["Neighborhood"] == "Buckhead Village")
           |(df_reduced["Neighborhood"] == "North Buckhead") |(df_reduced["Neighborhood"] == "Buckhead Heights")
          ]
df_buckhead

,Latitude,Longitude,Neighborhood
13,33.851150,-84.379440,Buckhead Forest
76,33.837520,-84.375740,Buckhead Village
118,33.848830,-84.381740,Buckhead Forest
217,33.847090,-84.372950,North Buckhead
226,33.847120,-84.373850,Buckhead Forest
...,...,...,...
350055,33.852786,-84.362043,North Buckhead
350059,33.851598,-84.380612,Buckhead Forest
350082,33.839330,-84.376588,Buckhead Village
350127,33.842061,-84.373026,Buckhead Village


In [75]:
df_buckhead['Neighborhood']

13         Buckhead Forest
76        Buckhead Village
118        Buckhead Forest
217         North Buckhead
226        Buckhead Forest
                ...       
350055      North Buckhead
350059     Buckhead Forest
350082    Buckhead Village
350127    Buckhead Village
350129    Buckhead Village
Name: Neighborhood, Length: 10156, dtype: object

In [80]:
df_buckhead = df_buckhead.assign(Neighborhood = 'Buckhead' )
df_buckhead

,Latitude,Longitude,Neighborhood
13,33.851150,-84.379440,Buckhead
76,33.837520,-84.375740,Buckhead
118,33.848830,-84.381740,Buckhead
217,33.847090,-84.372950,Buckhead
226,33.847120,-84.373850,Buckhead
...,...,...,...
350055,33.852786,-84.362043,Buckhead
350059,33.851598,-84.380612,Buckhead
350082,33.839330,-84.376588,Buckhead
350127,33.842061,-84.373026,Buckhead


In [81]:
df_other_hoods  = df_reduced[(df_reduced["Neighborhood"] == "Midtown") | (df_reduced["Neighborhood"] == "Brookhaven")
          |(df_reduced["Neighborhood"] == "Downtown") | (df_reduced["Neighborhood"] == "Old Fourth Ward")]
df_other_hoods

,Latitude,Longitude,Neighborhood
1,33.753200,-84.392010,Downtown
9,33.762690,-84.397020,Downtown
15,33.752830,-84.366870,Old Fourth Ward
16,33.780800,-84.373340,Midtown
17,33.770230,-84.388150,Downtown
...,...,...,...
350126,33.783060,-84.379421,Midtown
350138,33.763927,-84.396356,Downtown
350145,33.768814,-84.366527,Old Fourth Ward
350156,33.785508,-84.383434,Midtown


In [85]:
df_5hoods = pd.concat([df_buckhead,df_other_hoods], ignore_index=True, sort=False)
df_5hoods

,Latitude,Longitude,Neighborhood
0,33.851150,-84.379440,Buckhead
1,33.837520,-84.375740,Buckhead
2,33.848830,-84.381740,Buckhead
3,33.847090,-84.372950,Buckhead
4,33.847120,-84.373850,Buckhead
...,...,...,...
65538,33.783060,-84.379421,Midtown
65539,33.763927,-84.396356,Downtown
65540,33.768814,-84.366527,Old Fourth Ward
65541,33.785508,-84.383434,Midtown
